<h1 align="center"><u>Explainability AI Project</u></h1>
<h3 align="center"><u>Clément MOLLY-MITTON  | Diane VERBECQ  |  Raphaël VIGNAL</u></h3>
<h3 align="center"><u>Paul ESCALIER</u></h3>

# Table of Contents
1. [Introduction](#introduction)
2. [Import](#imports)
3. [Data exploration and cleaning](#data-exploration-and-cleaning)
4. [Models and evaluation](#models-and-evaluation)     
5. [Conclusion](#conclusion)

# Introduction
***


# Imports
***

We import all the necessary librairy

In [85]:
from copy import deepcopy
import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import optuna

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay, f1_score

# Data Exploration and Cleaning
***

In [86]:
original_data = pd.read_csv(r'dataset\World Energy Consumption.csv', delimiter= ',')
display(original_data.info())
display(original_data.head(5))
display(original_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22012 entries, 0 to 22011
Columns: 129 entries, country to wind_share_energy
dtypes: float64(126), int64(1), object(2)
memory usage: 21.7+ MB


None

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN


,year,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
count,22012.000000,1.812300e+04,1.111300e+04,1747.000000,2687.000000,2302.000000,2767.000000,5275.000000,5527.000000,5492.000000,...,6849.000000,4035.000000,2413.000000,4639.000000,4744.000000,7065.000000,7996.000000,4065.000000,6886.000000,4101.000000
mean,1974.213247,1.053096e+08,3.585114e+11,46.091694,2.719074,133.558459,38.068209,64.716800,10.966685,1.954035,...,0.622750,0.210395,267.541206,7.314174,53.422938,62.811828,17.561153,232.108949,1.404910,0.651413
std,35.058653,4.640460e+08,2.411179e+12,274.616745,10.057018,264.707851,111.623148,201.043915,47.953561,5.274155,...,2.066186,0.643304,5940.944120,37.327922,273.255306,252.734217,101.693172,733.841324,4.489043,1.893820
min,1900.000000,1.833000e+03,1.642060e+08,-100.000000,-50.843000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-100.000000,-51.208000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1946.000000,1.712404e+06,1.365898e+10,-0.616500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.029000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1984.000000,6.991543e+06,4.167411e+10,8.179000,0.000000,15.482000,0.673000,0.146000,0.010000,0.062000,...,0.000000,0.000000,20.000000,0.000000,0.005000,0.000000,0.000000,0.051000,0.000000,0.003000
75%,2003.000000,2.557688e+07,1.744295e+11,26.508500,0.861500,165.266750,13.629500,34.289000,0.690000,1.577500,...,0.095000,0.018500,47.971000,0.315000,2.357750,2.101000,0.140000,45.523000,0.372000,0.268000
max,2022.000000,7.975105e+09,1.136302e+14,5659.328000,141.131000,2588.512000,1199.207000,2514.102000,677.570000,71.429000,...,40.000000,7.586000,242384.844000,665.413000,5487.600000,3219.852000,2139.230000,8422.012000,56.840000,26.157000


based on https://github.com/owid/energy-data/blob/master/owid-energy-codebook.csv we will keep only the relevant colulumn for us and we will clean the data

In [96]:
column_to_keep = [
    # Basic info per country
    "country",
    "year",
    "iso_code",
    "population",
    "gdp", # PIB

    # Total energy consumption
    "primary_energy_consumption",
    "energy_per_capita",
    "energy_per_gdp",

    # Part of energy in the total consumption
    "fossil_share_energy",
    "renewables_share_energy",
    "low_carbon_share_energy",

    # Consumption per energy 
    "coal_consumption",
    "oil_consumption",
    "gas_consumption",
    "biofuel_consumption",
    "hydro_consumption",
    "nuclear_consumption",
    "solar_consumption",
    "wind_consumption",

    # Electricity production per energy
    "electricity_generation",
    "fossil_electricity",
    "renewables_electricity",
    "nuclear_electricity",
    "hydro_electricity",
    "solar_electricity",
    "wind_electricity",

    # Carbon and gas emission
    "carbon_intensity_elec",
    "greenhouse_gas_emissions",
]

In [98]:
# Lets first check the pourcentage of null value per column
# null_per_column = (original_data.isnull().sum() / original_data.shape[0] ) * 100
# null_per_column
# We can see that we have a lot a column with more than 50% of null value. We will drop them.
# column_to_keep = null_per_column[null_per_column < 50]
# column_to_keep

In [100]:
df_clean = original_data[column_to_keep]
df_clean

,country,year,iso_code,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp,fossil_share_energy,renewables_share_energy,...,wind_consumption,electricity_generation,fossil_electricity,renewables_electricity,nuclear_electricity,hydro_electricity,solar_electricity,wind_electricity,carbon_intensity_elec,greenhouse_gas_emissions
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,368.65,295.75,72.90,0.0,50.37,0.00,0.0,500.231,184.41
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,397.19,320.51,76.68,0.0,54.26,0.00,0.0,499.358,198.34
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,422.82,346.83,75.99,0.0,53.32,0.00,0.0,505.652,213.80
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,447.15,371.44,75.71,0.0,53.28,0.00,0.0,513.698,229.70
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,484.94,407.92,77.02,0.0,52.88,0.00,0.0,520.910,252.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22007,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,51.809,3441.986,2.281,NaN,NaN,...,NaN,9.19,3.73,5.46,0.0,5.05,0.02,0.0,355.822,3.27
22008,Zimbabwe,2019,ZWE,15354606.0,NaN,46.120,3003.655,NaN,NaN,NaN,...,NaN,8.24,3.66,4.58,0.0,4.17,0.03,0.0,387.136,3.19
22009,Zimbabwe,2020,ZWE,15669663.0,NaN,41.997,2680.132,NaN,NaN,NaN,...,NaN,7.59,3.40,4.19,0.0,3.81,0.03,0.0,389.987,2.96
22010,Zimbabwe,2021,ZWE,15993525.0,NaN,42.145,2635.154,NaN,NaN,NaN,...,NaN,8.03,3.61,4.42,0.0,4.00,0.04,0.0,392.279,3.15


Now lets see for each country the number of null value

In [105]:
df_clean.isnull().sum()

country                           0
year                              0
iso_code                       5500
population                     3889
gdp                           10899
primary_energy_consumption     9424
energy_per_capita             11410
energy_per_gdp                14801
fossil_share_energy           17225
renewables_share_energy       17225
low_carbon_share_energy       17225
coal_consumption              16592
oil_consumption               16299
gas_consumption               16797
biofuel_consumption           19245
hydro_consumption             16533
nuclear_consumption           17615
solar_consumption             17329
wind_consumption              17268
electricity_generation        14821
fossil_electricity            15462
renewables_electricity        13989
nuclear_electricity           13496
hydro_electricity             13128
solar_electricity             14056
wind_electricity              14016
carbon_intensity_elec         16847
greenhouse_gas_emissions    

In [101]:
nb_null = df_clean.isnull().groupby(df_clean["country"]).sum().sum(axis=1)
nb_element = df_clean.groupby(df_clean["country"]).size() * df_clean.shape[1]
null_percentage = (nb_null / nb_element ) * 100
null_percentage

country
ASEAN (Ember)     60.714286
Afghanistan       74.297424
Africa            56.736353
Africa (EI)       30.110837
Africa (Ember)    60.714286
                    ...    
World             54.819977
Yemen             55.813953
Yugoslavia        86.141304
Zambia            74.297424
Zimbabwe          74.390244
Length: 306, dtype: float64

In [103]:
# Again we can see that we have a lot of null value for certain country so we will keep only the country with less 
# than 50% of null value
# line_to_keep = null_percentage[null_percentage < 20]
# line_to_keep
# df_clean = df_clean[df_clean["country"].isin(line_to_keep.index)].reset_index().drop("index", axis=1)
# df_clean

# Models and Evaluation
***

# Conclusion
***